# Tutorial: RNN with opportunity dataset

In [1]:
import sys
sys.path.append('../')

# general
import numpy as np
import pandas as pd
from progressbar import ProgressBar

# dataset
from actreco.datasets.opportunityc import Opportunity
from actreco.utils.generator import batch_generator

# modeling
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


### Dataset

In [2]:
train_set = Opportunity(userID='S1,S2,S3')
x_train_list = train_set.get('X')
y_train_list = train_set.get('y')

In [3]:
nb_modal_dim = 2
nb_modal = train_set.nb_modal
nb_out = train_set.nb_class

kw = 3
nb_filter_list = [50, 40, 20]
nb_in_filter = 1
drop_rate = 0.5
nb_unit = 400

batch_size = 10
l_sample = 30
interval = int(l_sample * 0.5)

nb_iter = 100
report_each = 10



In [4]:
gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=True, nb_iter=nb_iter, nb_timestep=2, interval=interval, categorical=False)
for batch_idx, (X, y) in enumerate(gen):
    batch_idx += 1
    tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
    ty = Variable(torch.LongTensor(y), requires_grad=False).cuda()

In [5]:
# test
conv = nn.Conv2d(1, 10, (1, 5))
x = tX.cpu()
y = ty.cpu()
n, t, c, w, h = x.size()
x = conv(x.contiguous().view(n*t, c, w, h))
_ ,c, w, h = x.size()
x = x.view(n, t, c, w, h)


In [6]:
rnn = nn.LSTM(10*113*26, 200)

print(x.size())
x = x.view(n, t, -1).permute(1, 0, 2)
print(x.size())
x, _= rnn(x)
print(x.size())
fc = nn.Linear(x.size(2), 18)
x = fc(x.view(t*n, -1)).view(t, n, -1)

torch.Size([10, 2, 10, 113, 26])
torch.Size([2, 10, 29380])
torch.Size([2, 10, 200])


In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
criterion(x[0], y[:, 0]) + 0

Variable containing:
 2.8822
[torch.FloatTensor of size 1]

In [9]:
y.cuda().permute(1, 0)

Variable containing:
   15    17    15    14    14     4    12    17    17    17
   15    17    15    14    14     4    12    17    17    17
[torch.cuda.LongTensor of size 2x10 (GPU 0)]

In [10]:
test_set = Opportunity(userID='S4')
x_test_list = test_set.get('X')
y_test_list = test_set.get('y')

In [11]:
class VanilaConvRNN(nn.Module):
    def __init__(self, rnn_type, l_sample, nb_modal, nb_in_filter, nb_filter_list, kw, nb_unit, nb_out):
        super(VanilaConvRNN, self).__init__()
        self.rnn_type = rnn_type
        self.l_sample = l_sample
        self.nb_modal = nb_modal
        self.nb_conv = len(nb_filter_list)
        self.nlayers = 1
        self.nb_unit = nb_unit
        
        if isinstance(kw, int):
            kw = [kw] * self.nb_conv
        assert len(nb_filter_list) == len(kw)
        nb_filter_list = [nb_in_filter] + nb_filter_list
        
        self.kw = kw
        self.nb_filter_list = nb_filter_list

        convs = []
        for i in range(self.nb_conv):
            convs.append(nn.Conv2d(nb_filter_list[i], nb_filter_list[i+1], (1, kw[i])))
        self.convs = nn.ModuleList(convs)
        self.rnn = getattr(nn, rnn_type)(self.conv_output_shape(), self.nb_unit, num_layers=self.nlayers)
        self.fc2 = nn.Linear(self.nb_unit, nb_out)
        
    def forward(self, x):
        hidden = self.init_hidden(x.size(0))
        for conv in self.convs:
            n, t, c, w, h = x.size()
            x = x.contiguous().view(n*t, c, w, h)
            x = F.max_pool2d(F.relu(conv(x)), (1, 2))
            dropout = nn.Dropout2d(0.5)
            _ ,c, w, h = x.size()
            x = x.view(n, t, c, w, h)
            # x = dropout(x)
        x = x.view(n, t, -1).permute(1, 0, 2)
        
        x, hidden = self.rnn(x)
        x = x.view(t*n, -1)
        x = self.fc2(x)
        x = x.view(t, n, -1)
        return x
    
    def conv_output_shape(self):
        length = self.l_sample
        for i in range(self.nb_conv):
            length = int((length - self.kw[i] + 1) /2)
        return length * self.nb_filter_list[-1] * self.nb_modal
    

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers, bsz, self.nb_unit).zero_()),
                    Variable(weight.new(self.nlayers, bsz, self.nb_unit).zero_()))
        else:
            return Variable(weight.new(self.nlayers, bsz, self.nb_unit).zero_())

In [12]:
nb_modal_dim = 2
nb_modal = train_set.nb_modal
nb_out = train_set.nb_class

kw = 3
nb_filter_list = [50, 40, 20]
nb_in_filter = 1
drop_rate = 0.5
nb_unit = 400

batch_size = 128
l_sample = 30
interval = int(l_sample * 0.5)
nb_timestep = 10

nb_iter = 10000
report_each = 10


model = VanilaConvRNN(rnn_type='LSTM', l_sample=l_sample, nb_modal=nb_modal, nb_in_filter=1, nb_filter_list=nb_filter_list, kw=kw, nb_unit=nb_unit, nb_out=nb_out).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

In [13]:
model.train()

p = ProgressBar(max_value=nb_iter)
gen = batch_generator(x_train_list, y_train_list, batch_size, l_sample, train=True, nb_iter=nb_iter, categorical=False, interval=interval, nb_timestep=nb_timestep
)
model.train()
for batch_idx, (X, y) in enumerate(gen):
    batch_idx += 1
    tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
    ty = Variable(torch.LongTensor(y), requires_grad=False).cuda()
    ty = ty.permute(1, 0)
    
    optimizer.zero_grad()
    output = model(tX)
    
    loss = 0
    for i in range(nb_timestep):
        loss += criterion(output[i], ty[i])
    loss /= nb_timestep
    loss.backward()
    optimizer.step()
    if batch_idx % report_each == 0:
        print("{} samples, {} batch \t Loss: {:.6f}".format(batch_idx * batch_size, batch_idx, loss.data[0]))
    p.update(batch_idx)
    

  0% (10 of 10000) |                      | Elapsed Time: 0:00:01 ETA:  0:29:03

1280 samples, 10 batch 	 Loss: 2.608874


  0% (20 of 10000) |                      | Elapsed Time: 0:00:03 ETA:  0:30:19

2560 samples, 20 batch 	 Loss: 1.693245


  0% (30 of 10000) |                      | Elapsed Time: 0:00:05 ETA:  0:30:48

...

1277440 samples, 9980 batch 	 Loss: 0.070346


 99% (9990 of 10000) |################### | Elapsed Time: 0:34:36 ETA:  0:00:01

1278720 samples, 9990 batch 	 Loss: 0.059309


100% (10000 of 10000) |###################| Elapsed Time: 0:34:38 ETA:  0:00:00

1280000 samples, 10000 batch 	 Loss: 0.071899


In [18]:
from sklearn import metrics

def test(model, x_list, y_list, l_sample, nb_iter=1, batch_size=2048):
    model.eval()
    test_gen = batch_generator(x_list, y_list, batch_size, l_sample, train=False, nb_iter=nb_iter, categorical=False, seed=0, interval=interval, nb_timestep=nb_timestep)
    y_list = []
    py_list = []
    for X, y in test_gen:
        tX = Variable(torch.from_numpy(X.astype('float32')), requires_grad=False).cuda()
        py = model(tX).max(-1)[1].data.cpu().numpy().reshape(-1)
        y_list.append(y.reshape(-1))
        py_list.append(py)
    
    y = np.concatenate(y_list)
    py = np.concatenate(py_list)
    print(metrics.accuracy_score(y, py))
    print(metrics.f1_score(y, py, average='macro'))
    print(metrics.precision_score(y, py, average=None))
    print(metrics.recall_score(y, py, average=None))

In [19]:
test(model, x_test_list, y_test_list, l_sample, nb_iter=1, batch_size=batch_size)

0.51484375
0.0519416491278
[ 0.          0.          0.          0.02325581  0.11111111  0.04761905
  0.          0.          0.          0.          0.          0.          0.
  0.09090909  0.          0.06410256  0.          0.66326531]
[ 0.          0.          0.          0.02777778  0.01538462  0.02040816
  0.          0.          0.          0.          0.          0.          0.
  0.08333333  0.          0.05208333  0.          0.7629108 ]


In [16]:
def length(y, class_id):
    class_len = []
    current_len = 0
    flag = False
    for _y in y:
        if _y[class_id] == 1:
            current_len += 1
            flag = True
        if _y[class_id] !=1 & flag:
            class_len.append(current_len)
            flag = False
            current_len = 0
    print(np.mean(class_len), np.std(class_len), len(class_len))
    return class_len

In [17]:
for i in range(17):
    length(y_train_list[0], i)

(171.0, 0.0, 1)
(146.0, 10.0, 2)
(129.0, 0.0, 1)
(146.5, 21.5, 2)
(100.42857142857143, 36.749566150430923, 7)
(78.285714285714292, 14.399546478119031, 7)
(94.200000000000003, 14.288456879593403, 5)
(79.799999999999997, 17.724559232883621, 5)
(72.0, 2.0, 2)
(58.5, 1.5, 2)
(74.0, 0.0, 1)
(64.0, 0.0, 1)
(60.333333333333336, 14.636332266733433, 3)
(71.666666666666671, 9.9777530313971763, 3)
(118.33333333333333, 82.090735706831666, 3)
(336.0, 168.56848262155452, 6)
(69.400000000000006, 13.469966592386189, 5)
